In [1]:
import sys
sys.path.insert(1, '/home/richard/nfmc_jax/')
import nfmc_jax
import arviz as az
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scipy
from sklearn.neighbors import KernelDensity
import jax
import jax.numpy as jnp
import chaospy

# 1d Gaussian likelihood and prior test for nfmc_jax

In [7]:
def log_like(x, mu=0, sigma=1):
    return -0.5 * jnp.log(2 * np.pi * sigma ** 2) - 0.5 * (x - mu) ** 2 / (2 * sigma ** 2)

def log_prior(x):
    return -0.5 * jnp.log(2 * np.pi * 5.0 ** 2) - 0.5 * (x) ** 2 / (2 * 5.0 ** 2)

key = jax.random.PRNGKey(100)
prior_samples = jax.random.multivariate_normal(key, mean=jnp.array([0.0]), 
                                               cov=jnp.array([[5.0]]), shape=(10,))

In [8]:
trace = nfmc_jax.opt_nfo(log_like, log_prior, prior_samples, log_like_args=(0, 1), 
                         pmap_backend=None, n0=10, iteration=5)

stage:  0
theta0:  [[ 0.7025566 ]
 [-1.5459814 ]
 [-0.6087517 ]
 [ 0.86115724]
 [ 1.7978467 ]
 [ 2.2860527 ]
 [ 0.7708193 ]
 [ 2.126035  ]
 [-0.8433591 ]
 [ 1.8386974 ]]
Initialization logZ: -1.374, ESS/N: 0.873, logZ_pq: -1.355 log mean loss: -9.788
Stage:  1
No weights in the fit


/home/richard/nfmc_jax/nfmc_jax/nfo/nfo.py:293: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448222085/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  self.nf_model = GIS(torch.from_numpy(np.asarray(self.samples)),
/home/richard/nfmc_jax/nfmc_jax/sinf/SIT.py:428: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /opt/conda/con

beta =  1
weighting fit


IndexError: The shape of the mask [10, 10] at index 1 does not match the shape of the indexed tensor [10, 1] at index 1

# Prior samples sanity check ...

In [3]:
n=1
n_iter=1 #sinf iterations 
n_prior = 10#number of sobol points
n_samples = n_prior #not used explicitly
n0=n_prior
N=n_samples
#get sobol sequence for initial samples
low,high=-1.0,1.0
dist1d = chaospy.Iid(chaospy.Uniform(lower=low,upper=high),n)
init_prior=dist1d
init_prior = np.atleast_2d(dist1d.sample(n_prior,rule='sobol')).T
# init_prior=None
transform=None#'ahh'#'ahh' #turn off pymc3 transform 
if(transform is not None):
    trname='t'
    #logit( (x-a) / (b-a) )
    init_prior = None#inv_logit(init_prior,low,high)#logit((init_prior-low) /(high-low))
else:
    trname='nt'
knots=None#100 #default None
bw=0.5#None#1.#None#0.5#1. #default 2.0
redraw=False #don't redraw samples at every iteration, my local change
layers=2
ktrunc=np.inf #currently SKIPPING
bw_use_pq=True #whether to use pq loss in determining bw
t_ess=0.5
g_AF=1
aN=int(2*N)#10
bN=1#2#int(N/2)
beta_max=1.0

In [5]:
print(np.shape(init_prior))

(10, 1)
